In [1]:
%cd ..

/mnt/SSD_Data/active_projects/transformer_to_lstm


In [2]:
import os

os.environ['SEED'] = "42"

import dataclasses
from pathlib import Path
import warnings

import nlp
import torch
import numpy as np
import torch.nn.functional as F
from transformers import (
    BertForSequenceClassification,
    DistilBertForSequenceClassification
)
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.model_selection import train_test_split

from pytorch_helper_bot import (
    BaseBot, MovingAverageStatsTrackerCallback,  CheckpointCallback,
    LearningRateSchedulerCallback, MultiStageScheduler, Top1Accuracy,
    LinearLR, Callback
)

try:
    from apex import amp
    APEX_AVAILABLE = True
except ModuleNotFoundError:
    APEX_AVAILABLE = False


In [3]:
CACHE_DIR = Path("cache/")
CACHE_DIR.mkdir(exist_ok=True)

In [4]:
class SST2Dataset(torch.utils.data.Dataset):
    def __init__(self, entries_dict):
        super().__init__()
        self.entries_dict = entries_dict
    
    def __len__(self):
        return len(self.entries_dict["label"])
    
    def __getitem__(self, idx):
        return (
            self.entries_dict["input_ids"][idx],
            self.entries_dict["attention_mask"][idx],
            self.entries_dict["label"][idx]
        )

In [5]:
train_dict, valid_dict, test_dict = torch.load(str(CACHE_DIR / "distill-dicts.jbl"))

In [6]:
# Instantiate a PyTorch Dataloader around our dataset
train_loader = torch.utils.data.DataLoader(SST2Dataset(train_dict), batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(SST2Dataset(valid_dict), batch_size=64, drop_last=False)
test_loader = torch.utils.data.DataLoader(SST2Dataset(test_dict), batch_size=64, drop_last=False)

In [7]:
distill_bert_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

In [8]:
distill_bert_model = distill_bert_model.cuda()

In [9]:
optimizer = torch.optim.Adam(distill_bert_model.parameters(), lr=2e-5, betas=(0.9, 0.99))

In [10]:
if APEX_AVAILABLE:
    distill_bert_model, optimizer = amp.initialize(
        distill_bert_model, optimizer, opt_level="O1"
    )

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [11]:
class DistillTop1Accuracy(Top1Accuracy):
    def __call__(self, truth, pred):
        truth = truth["label"]
        return super().__call__(truth, pred)

In [12]:
@dataclasses.dataclass
class SST2Bot(BaseBot):
    log_dir = CACHE_DIR / "logs"
    
    def __post_init__(self):
        super().__post_init__()
        self.loss_format = "%.6f"

    @staticmethod
    def extract_prediction(output):
        return output[0]

In [13]:
total_steps = len(train_loader) * 5

checkpoints = CheckpointCallback(
    keep_n_checkpoints=1,
    checkpoint_dir=CACHE_DIR / "distill_model_cache/",
    monitor_metric="loss"
)
lr_durations = [
    int(total_steps*0.2),
    int(np.ceil(total_steps*0.8))
]
break_points = [0] + list(np.cumsum(lr_durations))[:-1]
callbacks = [
    MovingAverageStatsTrackerCallback(
        avg_window=len(train_loader) // 8,
        log_interval=len(train_loader) // 10
    ),
    LearningRateSchedulerCallback(
        MultiStageScheduler(
            [
                LinearLR(optimizer, 0.01, lr_durations[0]),
                CosineAnnealingLR(optimizer, lr_durations[1])
            ],
            start_at_epochs=break_points
        )
    ),
    checkpoints
]
    
bot = SST2Bot(
    log_dir = CACHE_DIR / "distill_logs",
    model=distill_bert_model, 
    train_loader=train_loader,
    valid_loader=valid_loader, 
    clip_grad=10.,
    optimizer=optimizer, echo=True,
    criterion=torch.nn.CrossEntropyLoss(),
    callbacks=callbacks,
    pbar=False, use_tensorboard=False,
    use_amp=APEX_AVAILABLE,
    metrics=(Top1Accuracy(),)
)

[INFO][07/01/2020 22:08:40] SEED: 42
[INFO][07/01/2020 22:08:40] # of parameters: 66,955,010
[INFO][07/01/2020 22:08:40] # of trainable parameters: 66,955,010


In [14]:
print(total_steps)

bot.train(
    total_steps=total_steps,
    checkpoint_interval=len(train_loader) // 2
)
bot.load_model(checkpoints.best_performers[0][1])
checkpoints.remove_checkpoints(keep=0)

[INFO][07/01/2020 22:08:40] Optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.99)
    eps: 1e-08
    initial_lr: 2e-05
    lr: 2e-05
    weight_decay: 0
)
[INFO][07/01/2020 22:08:40] Batches per epoch: 1053
[INFO][07/01/2020 22:08:40] ====================Epoch 1====================


5265


[INFO][07/01/2020 22:08:50] Step   105 | loss 0.675354 | lr: 2.18e-06 | 0.099s per step
[INFO][07/01/2020 22:09:00] Step   210 | loss 0.535189 | lr: 4.15e-06 | 0.094s per step


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


[INFO][07/01/2020 22:09:10] Step   315 | loss 0.330805 | lr: 6.13e-06 | 0.094s per step


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0


[INFO][07/01/2020 22:09:20] Step   420 | loss 0.301018 | lr: 8.10e-06 | 0.095s per step
[INFO][07/01/2020 22:09:30] Step   525 | loss 0.278078 | lr: 1.01e-05 | 0.090s per step
[INFO][07/01/2020 22:09:30] Metrics at step 526:
[INFO][07/01/2020 22:09:30] loss: 0.270641
[INFO][07/01/2020 22:09:30] accuracy: 87.84%
[INFO][07/01/2020 22:09:40] Step   630 | loss 0.264246 | lr: 1.21e-05 | 0.097s per step
[INFO][07/01/2020 22:09:49] Step   735 | loss 0.243720 | lr: 1.40e-05 | 0.090s per step
[INFO][07/01/2020 22:09:59] Step   840 | loss 0.237302 | lr: 1.60e-05 | 0.092s per step
[INFO][07/01/2020 22:10:09] Step   945 | loss 0.222996 | lr: 1.80e-05 | 0.094s per step
[INFO][07/01/2020 22:10:18] Step  1050 | loss 0.201085 | lr: 2.00e-05 | 0.092s per step
[INFO][07/01/2020 22:10:19] Metrics at step 1052:
[INFO][07/01/2020 22:10:19] loss: 0.261985
[INFO][07/01/2020 22:10:19] accuracy: 90.37%
[INFO][07/01/2020 22:10:20] ====================Epoch 2====================
[INFO][07/01/2020 22:10:29] Step 

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


[INFO][07/01/2020 22:15:48] Step  4620 | loss 0.043089 | lr: 1.14e-06 | 0.091s per step
[INFO][07/01/2020 22:15:58] Step  4725 | loss 0.044284 | lr: 8.05e-07 | 0.092s per step
[INFO][07/01/2020 22:15:59] Metrics at step 4734:
[INFO][07/01/2020 22:15:59] loss: 0.368667
[INFO][07/01/2020 22:15:59] accuracy: 89.22%
[INFO][07/01/2020 22:16:08] Step  4830 | loss 0.048006 | lr: 5.25e-07 | 0.094s per step
[INFO][07/01/2020 22:16:17] Step  4935 | loss 0.048634 | lr: 3.04e-07 | 0.091s per step
[INFO][07/01/2020 22:16:27] Step  5040 | loss 0.051134 | lr: 1.42e-07 | 0.091s per step
[INFO][07/01/2020 22:16:36] Step  5145 | loss 0.047391 | lr: 4.08e-08 | 0.091s per step
[INFO][07/01/2020 22:16:46] Step  5250 | loss 0.043795 | lr: 7.13e-10 | 0.091s per step
[INFO][07/01/2020 22:16:47] Metrics at step 5260:
[INFO][07/01/2020 22:16:47] loss: 0.364545
[INFO][07/01/2020 22:16:47] accuracy: 89.45%
[INFO][07/01/2020 22:16:48] Training finished. Best step(s):
[INFO][07/01/2020 22:16:48] loss: 0.261985 @ st

In [15]:
bot.eval(valid_loader)

{'loss': (0.26198503566444464, '0.261985'),
 'accuracy': (-0.9036697247706422, '90.37%')}

In [16]:
bot.eval(test_loader)

{'loss': (0.22761504229055632, '0.227615'),
 'accuracy': (-0.9128440366972477, '91.28%')}